In [ ]:
# セル0: Colabで yamagk2017@gmail.com のOAuth認証
from google.colab import auth
import gspread
import google.auth # google.auth モジュールをインポート

# OAuth認証
# このセルを実行すると、ブラウザで認証フローが開始され、
# yamagk2017@gmail.com でログインし、必要な権限を許可するよう求められます。
auth.authenticate_user()

# google.auth.default() から認証情報オブジェクトを取得
# これがgspreadが使用する認証情報になります。
credentials, project = google.auth.default()

# gspreadクライアントを認証
# gspread.authorize() を使用し、引数としてcredentialsを渡します。
client = gspread.authorize(credentials) # 修正点: gspread.authorize() を使用し、credentialsを直接渡す

In [ ]:
# セル1: 認証とSpotify設定
import pandas as pd
import requests

# Spotify設定（access_token は有効なものを使用）
access_token = "BQBj12ld4u9LqiQXkQYPeyR9kSFyHPJlzi_v_FkozqO1mS2USf5kM9iWYpO9hk6j_O0-ltOvS38cMVFMbIWa679PeGCB5lEuYmVgxkyDPB2ZpRzeqlPmh9n52JWTav60X6ALGyfsV4IoEEmsAzvFTXd0RX25jWaTm6R6xrApgD8i7ojaG-z9GD18Hr7YOOFG1ZHRuo6zVwLV54sQmg9tp2mBAxB0whjUgYI0fwH6rb2S8TUEb9uAii41ZHMKx3tGLsLoKv6mFnkjF-six6KGvBDgtNBt"
user_id = "12164908418"


In [ ]:
# セル2: 最新ワークシート名と週ラベル取得
sheet_name = "IHC_Weekly_Ranking"
sheet = client.open(sheet_name)
worksheet = sheet.worksheets()[-1]
worksheet_title = worksheet.title

def convert_sheet_name_to_label(sheet_name):
    try:
        year = sheet_name[0:4]
        start_month = int(sheet_name[5:7])
        start_day = int(sheet_name[7:9])
        end_month = int(sheet_name[11:13])
        end_day = int(sheet_name[13:15])
        start = f"{year}/{start_month}/{start_day}"
        end = f"{year}/{end_month}/{end_day}"
        return f"({start}-{end})"
    except Exception as e:
        print("❌ シート名の形式が正しくありません（例: 2025-0602to0608）")
        print("  実際のシート名:", sheet_name)
        raise e

week_range_label = convert_sheet_name_to_label(worksheet_title)
print("📄 最新ワークシート名:", worksheet_title)
print("📅 集計期間ラベル:", week_range_label)


📄 最新ワークシート名: 2026-0113to0119
📅 集計期間ラベル: (2026/1/13-2026/1/19)


In [ ]:
# セル3: 上位20アーティストのSpotify IDを取得
df = pd.DataFrame(worksheet.get_all_records())
top_20_ids = df["Spotify ID"].head(20).tolist()


In [ ]:

def get_latest_track_uri(artist_id):
    headers = {"Authorization": f"Bearer {access_token}"}
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    params = {
        "include_groups": "single,album",
        "market": "JP",
        "limit": 20
    }
    res = requests.get(url, headers=headers, params=params)
    albums = res.json().get("items", [])

    latest_album = None
    latest_date = ""

    for album in albums:
        release_date = album.get("release_date", "")
        if release_date > latest_date:
            latest_date = release_date
            latest_album = album

    if latest_album:
        album_id = latest_album["id"]
        tracks_url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
        track_res = requests.get(tracks_url, headers=headers)
        tracks = track_res.json().get("items", [])
        if tracks:
            return tracks[0]["uri"]
    return None

track_uris = []
for artist_id in top_20_ids:
    uri = get_latest_track_uri(artist_id)
    if uri:
        track_uris.append(uri)


In [ ]:
# セル5: プレイリスト取得 or 作成（デバッグ付き）
def get_or_create_playlist(user_id, playlist_name):
    # Ensure access_token is still valid before making API calls
    # You may need to implement a token refresh mechanism if your workflow requires it
    headers = {"Authorization": f"Bearer {access_token}"}
    res = requests.get(f"https://api.spotify.com/v1/users/{user_id}/playlists", headers=headers)
    playlists = res.json().get("items", [])
    for playlist in playlists:
        if playlist["name"].startswith(playlist_name):
            return playlist["id"]
    # 作成
    body = {
        "name": playlist_name,
        "description": "イマキテ週間ランキングTOP20 自動生成プレイリスト",
        "public": False
    }
    create_res = requests.post(
        f"https://api.spotify.com/v1/users/{user_id}/playlists",
        headers=headers,
        json=body
    )
    if create_res.status_code != 201:
        print("❌ プレイリスト作成に失敗しました")
        print("ステータスコード:", create_res.status_code)
        print("レスポンス:", create_res.json())
        raise Exception("Spotifyプレイリスト作成失敗")
    return create_res.json()["id"]

playlist_base_name = "イマキテ週間ランキングTOP20"
playlist_id = get_or_create_playlist(user_id, playlist_base_name)

In [ ]:
# セル6: プレイリスト更新
def replace_playlist_tracks(playlist_id, uris):
    headers = {"Authorization": f"Bearer {access_token}", "Content-Type": "application/json"}
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    requests.put(url, headers=headers, json={"uris": uris})

replace_playlist_tracks(playlist_id, track_uris)


In [ ]:
# セル7: プレイリスト名と説明の更新
def update_playlist_name(playlist_id, new_name, description):
    headers = {"Authorization": f"Bearer {access_token}", "Content-Type": "application/json"}
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    body = {
        "name": new_name,
        "description": description
    }
    requests.put(url, headers=headers, json=body)

new_playlist_name = f"{playlist_base_name} {week_range_label}"
update_playlist_name(playlist_id, new_playlist_name, f"イマキテランキング集計期間：{week_range_label}")


In [ ]:
# セル8: 完了メッセージ
print(f"✅ プレイリスト作成・更新完了: https://open.spotify.com/playlist/{playlist_id}")


✅ プレイリスト作成・更新完了: https://open.spotify.com/playlist/1vkTUuGlIHdZmurmWg7YOF
